Dates Trial

In [48]:
import pandas as pd
from sqlalchemy import create_engine, select, delete, insert, update
import config

import dateutil.parser
import os
from glob import glob
from tqdm import tqdm

def extract_begin_end_dates(date_range):
    if '-' not in date_range:
        # 'July 30, 2012'
        return dateutil.parser.parse(date_range), dateutil.parser.parse(date_range)
    else:
        date_range = date_range.replace(',', '')
        try: # 'July 30-31, 2012'
            month, dates, year = date_range.split(' ')
            dates = dates.split('-')
            begin_date = dateutil.parser.parse(' '.join([dates[0], month, year]))
            end_date = dateutil.parser.parse(' '.join([dates[1], month, year]))
        except: # 'July 31-August 1, 2012'
            date_range = date_range.replace('-', ' ')
            begin_date_month, bigin_date_date, end_date_month, end_date_date, year = date_range.split(' ')
            begin_date = dateutil.parser.parse(' '.join([bigin_date_date, begin_date_month, year]))
            end_date = dateutil.parser.parse(' '.join([end_date_date, end_date_month, year]))
            
        # April/May 30-1, 2013
        # Jan/Feb 31-1, 2017
        # Oct/Nov 31-1, 2017
        # Jul/Aug 31-1, 2018
        # April/May 30-1, 2019
        return begin_date, end_date

date = 'April/May 30-1, 2019'

b, e = extract_begin_end_dates(date)
print(b)
print(e)

2012-07-31 00:00:00
2012-08-01 00:00:00


In [85]:
date_range='July 31-August 1, 2012'
date_range = date_range.replace(',', '')
# month, days, year = date_range.split(' ')

date_range = date_range.replace('-', ' ')
# days.split('-')
date_range.split(' ')

['July', '31', 'August', '1', '2012']

In [77]:
date_range.replace(',', '')

'July 31-August 1 2012'

In [40]:
string = 'April/May 30-1, 2013'

begin_month, end_month, begin_date, end_date, year = string.replace(',', '').replace('-', ' ').replace('/', ' ').split(' ')
new = f'{begin_month} {begin_date}-{end_month} {end_date}, {year}'

['April', 'May', '30', '1', '2013']


'April 30-May 1, 2013'

In [53]:
from daterangeparser import parse

def extract_begin_end_dates(date_range):
    if '-' not in date_range: # 'July 30, 2012'
        parsed, _ = parse(date_range)
        return parsed, parsed
    
    elif '/' in date_range: # 'April/May 30-1, 2013'
        begin_month, end_month, begin_date, end_date, year = date_range.replace(',', '').replace('-', ' ').replace('/', ' ').split(' ')
        date_range = f'{begin_month} {begin_date}-{end_month} {end_date}, {year}'
        return parse(date_range)
        
    else: # 'July 30-31, 2012', 'July 31-August 1, 2012'
        return parse(date_range)

for string in ['July 30, 2012', 'July 30-31, 2012', 'July 31-August 1, 2012', 'April/May 30-1, 2013']:
    start, end = extract_begin_end_dates(string)
    print(start)
    print(end)
    print('-'*50)

2012-07-30 00:00:00
2012-07-30 00:00:00
--------------------------------------------------
2012-07-30 00:00:00
2012-07-31 00:00:00
--------------------------------------------------
2012-07-31 00:00:00
2012-08-01 00:00:00
--------------------------------------------------
2013-04-30 00:00:00
2013-05-01 00:00:00
--------------------------------------------------


Minio Trial 

http://147.46.94.186:40911/login

In [38]:
from minio import Minio
from minio.error import S3Error
import os
import argparse
import config

import os

In [90]:

# # Define arguments
# parser = argparse.ArgumentParser(description='MinIO Data Uploader')
# parser.add_argument('-b', '--bucket_name', type=str, default='disclosures',
#                     help='Bucket name')
# parser.add_argument('-fn', '--file_name', type=str, required=True,
#                     help='(REQUIRED) name of the file to be uploaded')
# parser.add_argument('-fp', '--file_path', type=str, required=True,
#                     help='(REQUIRED) object name to be used as file path')

# args = parser.parse_args()

bucket_name = 'disclosures'
file_name = './dataframe.parquet'
file_path = '/FOMC/2022/dataframe.parquet'

# Get Minio Access Info
ACCESS_KEY = config.user
SECRET_KEY = config.pw

# Declare the endpoint
endpoint='147.46.94.186:40911'
# endpoint = "snu-bdai.minio"

##########
client = Minio(
    endpoint,
    access_key=ACCESS_KEY,
    secret_key=SECRET_KEY, secure=False
)

# Create bucket if it does not exist
if not client.bucket_exists(bucket_name):
    client.make_bucket(bucket_name)
    print("Created bucket {}".format(bucket_name))

# Upload Data
client.fput_object(
    bucket_name, file_name, file_path,
)
print(f"Uploaded {file_name} to: {file_path}")

# if __name__ == "__main__":
#     try:
#         main()
#     except S3Error as exc:
#         print("S3Error! ", exc)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\urllib3\util\connection.py", line 84, in create_connection
    raise err
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\urllib3\util\connection.py", line 74, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Jihye Park\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 392, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "C:\Users\Jihye Park\anaconda3\lib\http\client.py", line 1255